In [8]:
from google.cloud import storage, bigquery
import os
from datetime import datetime

# Caminho para o arquivo de credenciais JSON
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\diego\OneDrive\DATAMASTER\dados_bacen_new\case-radar-de-risco-f1ef8b6de88c.json"

# 📅 Data atual no formato YYYYMMDD
data_hoje = datetime.today().strftime('%Y%m%d')

# 🔧 Parâmetros dinâmicos
nome_arquivo = f"inadimplencia_estimada_uf_{data_hoje}.csv"
local_file_path = fr"C:\Users\diego\OneDrive\DATAMASTER\dados_bacen_new\{nome_arquivo}"
bucket_name = "bucket-datamaster"
destination_blob_name = nome_arquivo
dataset_id = "camada_bronze"
table_id = "inadimplencia_estimada_uf"

# 1️⃣ Upload para o bucket no GCS
def upload_to_bucket():
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(local_file_path)
    print(f"✅ Arquivo {local_file_path} enviado para gs://{bucket_name}/{destination_blob_name}")

# 2️⃣ Importar para o BigQuery
def import_to_bigquery():
    client = bigquery.Client()

    uri = f"gs://{bucket_name}/{destination_blob_name}"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
    )

    table_ref = client.dataset(dataset_id).table(table_id)
    load_job = client.load_table_from_uri(uri, table_ref, job_config=job_config)

    load_job.result()  # Espera o job terminar
    print(f"✅ Tabela {table_id} criada no BigQuery a partir de {uri}")

# 🚀 Executa as funções
upload_to_bucket()
import_to_bigquery()

✅ Arquivo C:\Users\diego\OneDrive\DATAMASTER\dados_bacen_new\inadimplencia_estimada_uf_20250815.csv enviado para gs://bucket-datamaster/inadimplencia_estimada_uf_20250815.csv
✅ Tabela inadimplencia_estimada_uf criada no BigQuery a partir de gs://bucket-datamaster/inadimplencia_estimada_uf_20250815.csv
